<a href="https://colab.research.google.com/github/diggs1711/comma-ai-speed-challenge/blob/master/speed_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
from cv2 import VideoCapture
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import gc
import math

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from pathlib import Path
drive_path = Path('/content/drive/My Drive/comma_ai_speed_challenge/speedchallenge-master/data/')

In [0]:
train_vals = np.array(pd.read_fwf(drive_path/'train.txt', header=None))

In [4]:
print(f"mean {train_vals.mean()}")
print(f"max {train_vals.max()}")
print(f"std {train_vals.std()}")
print(f"min {train_vals.min()}")

idx = int(len(train_vals) / 2)
print()
print("*** First half of video ****")
first_half = train_vals[:idx]
print(f"mean 1st {first_half.mean()}")
print(f"max 1st {first_half.max()}")
print(f"std 1st {first_half.std()}")
print(f"min 1st {first_half.min()}")

print()
print("*** Second half of video ****")
second_half = train_vals[idx:]
print(f"mean 2nd {second_half.mean()}")
print(f"max 2nd {second_half.max()}")
print(f"std 2nd {second_half.std()}")
print(f"min 2nd {second_half.min()}")

mean 12.183181660441177
max 28.130404
std 8.206561706745727
min 0.0

*** First half of video ****
mean 1st 18.66717958362745
max 1st 28.130404
std 1st 5.883530076403356
min 1st 0.0

*** Second half of video ****
mean 2nd 5.699183737254902
max 2nd 14.890168
std 2nd 3.9993656743963695
min 2nd 0.0


In [0]:
numberOfFrame = 20400
fps = 20
cropped_image_height = 100
cropped_image_width = 440

In [0]:
# load video frame by frame
# cap = VideoCapture(str(drive_path/'train.mp4'))
# i = 0
# video_frames = []

# while i < numberOfFrame:
#   ret_val, frame = cap.read()
#   cropped_image = frame[200:200+cropped_image_height,100:100+cropped_image_width, 2]
#   video_frames.append(np.array(cropped_image))
#   i = i + 1

# cap.release()

In [0]:
# video_frames = np.array(video_frames)

In [0]:
# np.save(str(drive_path/'video_frames'), video_frames)

In [0]:
video_frames = np.load(str(drive_path/'video_frames.npy')) / 255

In [0]:
def chunk_video(data, target_data=None):
  chunk_size = 100
  result = []
  target = []
  i = 1
  while i < len(data):
    if i % chunk_size == 0:
      result.append(data[i-chunk_size:i])
      if target_data.any():
        target.append(target_data[i-chunk_size:i])
    i = i + 1

  return result, target

In [0]:
X_train, y_train = chunk_video(video_frames, train_vals)

In [0]:
# create train and validation sets
from sklearn.utils import shuffle

valid_idx =  180
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train, y_train = shuffle(X_train, y_train, random_state=1)

X_valid = X_train[valid_idx:]
y_valid = y_train[valid_idx:]

X_train = X_train[:valid_idx]
y_train = y_train[:valid_idx]

In [15]:
del video_frames, train_vals;
gc.collect()

0

In [0]:
X_train = X_train[..., np.newaxis]
y_train = y_train

X_valid = X_valid[..., np.newaxis]
y_valid = y_valid

In [17]:
X_train.shape

(180, 100, 100, 440, 1)

In [18]:
y_train.shape

(180, 100, 1)

In [19]:
from keras.layers import Dense, Flatten, TimeDistributed, SimpleRNN, LSTM, Bidirectional, BatchNormalization, Conv2D, Dropout, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, concatenate, Reshape
from keras.models import Sequential
from keras import Model, Input
import keras

def build_model():
  img_input = Input(shape=(None, cropped_image_height, cropped_image_width, 1))

  x = TimeDistributed(Conv2D(16, 3, activation='relu'))(img_input)
  x = TimeDistributed(BatchNormalization())(x)
  x = TimeDistributed(GlobalMaxPooling2D())(x)

  x = TimeDistributed(Dense(100, activation='relu'))(x)
  x = Bidirectional(LSTM(4, return_sequences=True))(x)
  x = Bidirectional(LSTM(4, return_sequences=True))(x)
  x = TimeDistributed(Dense(20, activation='relu'))(x)

  x = TimeDistributed(Dense(1, activation='relu'))(x)
  
  mod = Model(inputs=[img_input], outputs=x)
  mod.compile('adam', 'mse')
  return mod

Using TensorFlow backend.


In [20]:
model = build_model()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 100, 440, 1) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 98, 438, 16) 160       
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 98, 438, 16) 64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 16)          0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 100)         1700      
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 8)           3360      
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 8)           416 

In [0]:
X, y = shuffle(X_train, y_train, random_state=42)
X_val, y_val = shuffle(X_valid, y_valid, random_state=42)

In [0]:
history = model.fit(
    X,
    y, 
    epochs=200,
    batch_size=1,
    validation_data=(X_valid, y_valid))

Train on 180 samples, validate on 23 samples
Epoch 1/200
180/180 [==============================] - 158s 879ms/step - loss: 170.7253 - val_loss: 97.5046
Epoch 2/200
180/180 [==============================] - 150s 832ms/step - loss: 78.8513 - val_loss: 79.3212
Epoch 3/200
180/180 [==============================] - 149s 827ms/step - loss: 66.7621 - val_loss: 83.7779
Epoch 4/200
166/180 [==========================>...] - ETA: 11s - loss: 67.2847

In [0]:
def load_video_and_save_to_drive(filename, numberOfFrames):
  # load video frame by frame
  cap = VideoCapture(str(drive_path/filename))
  i = 0
  video_frames = []

  while i < numberOfFrames:
    ret_val, frame = cap.read()
    cropped_image = frame[200:200+cropped_image_height,100:100+cropped_image_width, 2]
    video_frames.append(np.array(cropped_image))
    i = i + 1

  print("processed finished")
  video_frames = np.array(video_frames)
  print("Saving...")
  np.save(str(drive_path/f'frames-{filename}'), video_frames)
  cap.release()

In [116]:
load_video_and_save_to_drive('test.mp4', 10798)

processed finished
Saving...


In [0]:
test_video_frames = np.load(str(drive_path/'frames-test.mp4.npy'))

In [28]:
test_video_frames.shape

(10798, 100, 440)

In [0]:
test_video_frames = test_video_frames[np.newaxis, ..., np.newaxis] / 255

In [31]:
predictions = model.predict(test_video_frames[:500])

ResourceExhaustedError: ignored

In [81]:
predictions

array([[17.770994],
       [18.333986],
       [19.05152 ],
       ...,
       [14.621207],
       [14.47956 ],
       [14.278064]], dtype=float32)

In [82]:
train_vals.min()

0.0

In [83]:
predictions.min()

3.986187

In [84]:
predictions.max()

25.80058

In [79]:
predictions.mean()

14.727596